In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

In [79]:
train = pd.read_csv('./train.csv', sep='\t')
test = pd.read_csv('./test.csv', sep='\t')

In [3]:
train.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,336,337,338,339,340,341,342,343,344,345
0,0,1,1,0,0,0,0,0,0.090909,0,...,0.221395,0,1,0,0,0.222222,1,1,1,1
1,1,1,1,0,0,1,0,0,0.090909,0,...,0.241508,0,1,0,0,0.111111,1,1,1,0
2,2,0,1,0,0,1,0,0,0.090909,0,...,0.123067,0,1,0,0,0.444444,1,1,1,1
3,3,0,1,0,0,1,0,0,0.136364,0,...,0.296065,0,0,1,0,0.222222,1,1,1,0
4,4,0,1,0,0,1,0,0,0.136364,0,...,0.178956,0,0,1,0,0.111111,1,1,1,1


In [106]:
features = list(train.columns)

#remove index and target
features.remove('Unnamed: 0')
features.remove('0')
features.remove('160')
features.remove('164')

print('Length of features {}'.format(len(features)))

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [81]:
X = train[features] 
y = train['0']

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

## XGBClassifier

Validation : 0.73648  
LB : 0.74997968

In [6]:
parameters = {
    #General Parameters
    'silent': 1,
    "nthread": 4,
    
    #Booster Parameters
    'eta': 0.1,
    'max_depth': 4,
    'min_child_weight': 5,
    'gamma': 0.1,
    'subsample': 1,
    'alpha': 0,
    'lambda': 1,
    'colsample_bytree': 1,
    
    #Learning Task Parameters
    'objective': 'reg:logistic',
    "seed": 42,
    "eval_metric": 'auc'
}


dmatrix_train = xgb.DMatrix(X, y, feature_names=X.columns)

res_cv = xgb.cv(
    params=parameters, 
    dtrain=dmatrix_train, 
    num_boost_round=1000, 
    early_stopping_rounds=10,
    folds=skf,
    verbose_eval=1
)


[0]	train-auc:0.673927+0.00575216	test-auc:0.665735+0.00971735
[1]	train-auc:0.69146+0.00429029	test-auc:0.680147+0.00515784
[2]	train-auc:0.69648+0.00279376	test-auc:0.683875+0.0090845
[3]	train-auc:0.698646+0.000911512	test-auc:0.686536+0.00669547
[4]	train-auc:0.701905+0.000652304	test-auc:0.68972+0.00540706
[5]	train-auc:0.70608+0.00125098	test-auc:0.692361+0.00790332
[6]	train-auc:0.708894+0.00176114	test-auc:0.694384+0.0079265
[7]	train-auc:0.714134+0.00353586	test-auc:0.697896+0.00912097
[8]	train-auc:0.716497+0.0036014	test-auc:0.699735+0.00872768
[9]	train-auc:0.719117+0.00264675	test-auc:0.701625+0.0074002
[10]	train-auc:0.721022+0.00174992	test-auc:0.702601+0.00662604
[11]	train-auc:0.72341+0.00135421	test-auc:0.704816+0.00606212
[12]	train-auc:0.725644+0.000935907	test-auc:0.706232+0.00602586
[13]	train-auc:0.727054+0.000475003	test-auc:0.707779+0.00577014
[14]	train-auc:0.729824+0.000811524	test-auc:0.708819+0.00591734
[15]	train-auc:0.731751+0.000497333	test-auc:0.710027+

[129]	train-auc:0.824123+0.00310362	test-auc:0.73552+0.00192335
[130]	train-auc:0.824671+0.00322605	test-auc:0.735638+0.00166255
[131]	train-auc:0.825054+0.00321734	test-auc:0.735657+0.0015219
[132]	train-auc:0.825576+0.00325091	test-auc:0.73556+0.00151044
[133]	train-auc:0.826236+0.00334156	test-auc:0.735736+0.00147842
[134]	train-auc:0.826898+0.00327154	test-auc:0.735992+0.00146869
[135]	train-auc:0.827278+0.00323933	test-auc:0.735999+0.00154843
[136]	train-auc:0.827496+0.00334763	test-auc:0.735981+0.00155972
[137]	train-auc:0.827715+0.00329415	test-auc:0.735971+0.00157959
[138]	train-auc:0.82828+0.00347431	test-auc:0.73611+0.00170833
[139]	train-auc:0.82868+0.00351384	test-auc:0.736131+0.00184169
[140]	train-auc:0.828856+0.00355084	test-auc:0.736161+0.0018125
[141]	train-auc:0.829063+0.00351211	test-auc:0.736123+0.00181427
[142]	train-auc:0.829411+0.00338972	test-auc:0.73616+0.00178499
[143]	train-auc:0.829871+0.00349978	test-auc:0.736248+0.00174693
[144]	train-auc:0.830367+0.003600

In [82]:
parameters = {
    #General Parameters
    'silent': 1,
    "nthread": 4,
    
    #Booster Parameters
    'learning_rate': 0.1,
    'max_depth': 4,
    'min_child_weight': 5,
    'gamma': 0.1,
    'subsample': 1,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'colsample_bytree': 1,
    
    #Learning Task Parameters
    'objective': 'reg:logistic',
    "seed": 42,
    "eval_metric": 'auc'
}


model = XGBClassifier(**parameters)

In [83]:
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=0.1, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=5, missing=None,
       n_estimators=100, n_jobs=1, nthread=4, objective='reg:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=42, silent=1, subsample=1)

In [84]:
y_pred_xgb_model = model.predict_proba(test[features])[:, 1]

In [ ]:
test_df = test[['Unnamed: 0', '0']]

test_df['0'] = y_pred_xgb_model

test_df.to_csv('XGB_tunned.csv', index=False)

## LightGBM

Validation : 0.74012324  
LB : 0.76086891


In [91]:
import lightgbm as lgb


lgb_train = lgb.Dataset(X, label=y, free_raw_data=False)

In [31]:
parameters = {
    'objective': 'binary',
    'learning_rate': 0.01,
    'max_depth': 8,
    'num_threads': 4,
    'metric': 'auc',
    'n_estimators': 580,
    
        #regularization
    'colsample_bytree': 0.6,
    'subsample': 1,
    'subsample_freq': 2,
    'min_data_in_leaf': 20,
}

n_rounds = 10000

result = lgb.cv(params=parameters, 
                train_set=lgb_train, 
#                 num_boost_round=n_rounds, 
                folds=skf.split(X, y), 
                early_stopping_rounds=10, 
                verbose_eval=1, 
)

[1]	cv_agg's auc: 0.667718 + 0.00188744
[2]	cv_agg's auc: 0.682796 + 0.00298733
[3]	cv_agg's auc: 0.688896 + 0.00380545
[4]	cv_agg's auc: 0.690268 + 0.00165038
[5]	cv_agg's auc: 0.695486 + 0.00282954
[6]	cv_agg's auc: 0.699334 + 0.00452896
[7]	cv_agg's auc: 0.700075 + 0.00443784
[8]	cv_agg's auc: 0.702796 + 0.00514051
[9]	cv_agg's auc: 0.704223 + 0.00454418
[10]	cv_agg's auc: 0.706173 + 0.00448215
[11]	cv_agg's auc: 0.706681 + 0.00381225
[12]	cv_agg's auc: 0.707389 + 0.00388509
[13]	cv_agg's auc: 0.707274 + 0.00404577
[14]	cv_agg's auc: 0.707673 + 0.0040165
[15]	cv_agg's auc: 0.707822 + 0.00410791
[16]	cv_agg's auc: 0.70788 + 0.00379943
[17]	cv_agg's auc: 0.708057 + 0.00351652
[18]	cv_agg's auc: 0.708512 + 0.00359139
[19]	cv_agg's auc: 0.708388 + 0.00356911
[20]	cv_agg's auc: 0.708226 + 0.00350477
[21]	cv_agg's auc: 0.708739 + 0.00331152
[22]	cv_agg's auc: 0.708861 + 0.00315153
[23]	cv_agg's auc: 0.708888 + 0.00308773
[24]	cv_agg's auc: 0.709093 + 0.00331191
[25]	cv_agg's auc: 0.709662

[201]	cv_agg's auc: 0.727412 + 0.00139115
[202]	cv_agg's auc: 0.727503 + 0.00140224
[203]	cv_agg's auc: 0.727615 + 0.00142113
[204]	cv_agg's auc: 0.727718 + 0.00145031
[205]	cv_agg's auc: 0.727874 + 0.0014375
[206]	cv_agg's auc: 0.72796 + 0.00138885
[207]	cv_agg's auc: 0.7281 + 0.00143348
[208]	cv_agg's auc: 0.728143 + 0.00142177
[209]	cv_agg's auc: 0.728154 + 0.00138799
[210]	cv_agg's auc: 0.728251 + 0.00139242
[211]	cv_agg's auc: 0.728349 + 0.00144541
[212]	cv_agg's auc: 0.728478 + 0.00142798
[213]	cv_agg's auc: 0.728593 + 0.00136017
[214]	cv_agg's auc: 0.72871 + 0.00134574
[215]	cv_agg's auc: 0.728826 + 0.00136992
[216]	cv_agg's auc: 0.728944 + 0.00136983
[217]	cv_agg's auc: 0.729006 + 0.00135056
[218]	cv_agg's auc: 0.729066 + 0.00131115
[219]	cv_agg's auc: 0.729138 + 0.00125965
[220]	cv_agg's auc: 0.729218 + 0.0012503
[221]	cv_agg's auc: 0.729333 + 0.00126792
[222]	cv_agg's auc: 0.729472 + 0.0013135
[223]	cv_agg's auc: 0.729545 + 0.00135027
[224]	cv_agg's auc: 0.729634 + 0.00137242

[400]	cv_agg's auc: 0.737669 + 0.00173691
[401]	cv_agg's auc: 0.73769 + 0.00178395
[402]	cv_agg's auc: 0.737746 + 0.00179025
[403]	cv_agg's auc: 0.737764 + 0.0017967
[404]	cv_agg's auc: 0.737768 + 0.00180421
[405]	cv_agg's auc: 0.737783 + 0.00176943
[406]	cv_agg's auc: 0.737826 + 0.0017723
[407]	cv_agg's auc: 0.737852 + 0.00176819
[408]	cv_agg's auc: 0.737894 + 0.00176362
[409]	cv_agg's auc: 0.737915 + 0.00176294
[410]	cv_agg's auc: 0.737947 + 0.00176746
[411]	cv_agg's auc: 0.737976 + 0.00178841
[412]	cv_agg's auc: 0.737996 + 0.00179376
[413]	cv_agg's auc: 0.738004 + 0.00180031
[414]	cv_agg's auc: 0.737993 + 0.00181615
[415]	cv_agg's auc: 0.738013 + 0.00181322
[416]	cv_agg's auc: 0.738018 + 0.00185944
[417]	cv_agg's auc: 0.738032 + 0.00185295
[418]	cv_agg's auc: 0.738057 + 0.00184076
[419]	cv_agg's auc: 0.738059 + 0.00184509
[420]	cv_agg's auc: 0.738117 + 0.00185048
[421]	cv_agg's auc: 0.73814 + 0.00186509
[422]	cv_agg's auc: 0.738209 + 0.00188595
[423]	cv_agg's auc: 0.738255 + 0.00185

In [94]:
def check_train_score(params, lgb_data, data, target, kf, num_rounds):
    roc_auc = []
    for train, val in kf.split(data,target):
        temp_lgb_train = lgb_data.subset(train)
        temp_lgb_val = lgb_data.subset(val)
        temp_model = lgb.train(params, temp_lgb_train, num_rounds, verbose_eval=num_rounds)
        roc_auc.append([roc_auc_score(target[train],temp_model.predict(data.loc[train])), roc_auc_score(target[val],temp_model.predict(data.loc[val]))])
    return np.mean(roc_auc, axis=0)


lgb_train = lgb.Dataset(X, label=y, free_raw_data=False)
check_train_score(parameters, lgb_train, X, y, skf, 580)

In [44]:
parameters = {
    'objective': 'binary',
    'learning_rate': 0.01,
    'max_depth': 8,
    'num_threads': 4,
    'metric': 'auc',
    'n_estimators': 580,
    
        #regularization
    'colsample_bytree': 0.6,
    'subsample': 1,
    'subsample_freq': 2,
    'min_data_in_leaf': 20,
}


model = LGBMClassifier(**parameters)

In [45]:
model.fit(X, y)

y_pred_lgb_model = model.predict_proba(test[features])[:, 1]

test_df = test[['Unnamed: 0', '0']]

test_df['0'] = y_pred_lgb_model

# test_df.to_csv('LGB_tunned.csv', index=False)

## CatBoost

Validation : 0.7390581327666667  
LB : 0.75077948


In [6]:
import catboost as ctb

ctb_data = ctb.Pool(X, y)

In [8]:
parameters = {
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 10000,
    'learning_rate': 0.03,
    'random_seed': 42,
    'use_best_model': False,
    'od_wait': 30,
    'od_type': 'Iter'
}

ctb.cv(ctb_data, parameters, stratified=True, seed=42, logging_level='Verbose')

0:	learn: 0.6505172	test: 0.6453778	best: 0.6453778 (0)	total: 650ms	remaining: 1h 48m 23s
1:	learn: 0.6588441	test: 0.6523765	best: 0.6523765 (1)	total: 1.47s	remaining: 2h 2m 24s
2:	learn: 0.6638678	test: 0.6594020	best: 0.6594020 (2)	total: 2.06s	remaining: 1h 54m 38s
3:	learn: 0.6680095	test: 0.6600565	best: 0.6600565 (3)	total: 2.72s	remaining: 1h 53m 18s
4:	learn: 0.6745568	test: 0.6644184	best: 0.6644184 (4)	total: 3.32s	remaining: 1h 50m 29s
5:	learn: 0.6785424	test: 0.6695447	best: 0.6695447 (5)	total: 3.99s	remaining: 1h 50m 44s
6:	learn: 0.6774839	test: 0.6684815	best: 0.6695447 (5)	total: 4.55s	remaining: 1h 48m 10s
7:	learn: 0.6798770	test: 0.6711742	best: 0.6711742 (7)	total: 5.22s	remaining: 1h 48m 39s
8:	learn: 0.6823285	test: 0.6734053	best: 0.6734053 (8)	total: 5.89s	remaining: 1h 49m
9:	learn: 0.6831554	test: 0.6739213	best: 0.6739213 (9)	total: 6.56s	remaining: 1h 49m 16s
10:	learn: 0.6828351	test: 0.6732216	best: 0.6739213 (9)	total: 7.33s	remaining: 1h 50m 51s
11:

90:	learn: 0.7282772	test: 0.7101779	best: 0.7101779 (90)	total: 1m 3s	remaining: 1h 55m 29s
91:	learn: 0.7285854	test: 0.7104916	best: 0.7104916 (91)	total: 1m 4s	remaining: 1h 55m 21s
92:	learn: 0.7289482	test: 0.7106536	best: 0.7106536 (92)	total: 1m 4s	remaining: 1h 55m 17s
93:	learn: 0.7295202	test: 0.7111355	best: 0.7111355 (93)	total: 1m 5s	remaining: 1h 55m 14s
94:	learn: 0.7299591	test: 0.7112448	best: 0.7112448 (94)	total: 1m 6s	remaining: 1h 55m 8s
95:	learn: 0.7303656	test: 0.7115167	best: 0.7115167 (95)	total: 1m 6s	remaining: 1h 55m 2s
96:	learn: 0.7307392	test: 0.7117498	best: 0.7117498 (96)	total: 1m 7s	remaining: 1h 54m 57s
97:	learn: 0.7311090	test: 0.7120377	best: 0.7120377 (97)	total: 1m 8s	remaining: 1h 54m 53s
98:	learn: 0.7312597	test: 0.7120697	best: 0.7120697 (98)	total: 1m 8s	remaining: 1h 54m 49s
99:	learn: 0.7316133	test: 0.7124178	best: 0.7124178 (99)	total: 1m 9s	remaining: 1h 54m 43s
100:	learn: 0.7318810	test: 0.7126692	best: 0.7126692 (100)	total: 1m 10

176:	learn: 0.7519955	test: 0.7231162	best: 0.7231162 (176)	total: 2m 2s	remaining: 1h 52m 54s
177:	learn: 0.7522841	test: 0.7231595	best: 0.7231595 (177)	total: 2m 2s	remaining: 1h 52m 54s
178:	learn: 0.7525547	test: 0.7232471	best: 0.7232471 (178)	total: 2m 3s	remaining: 1h 52m 51s
179:	learn: 0.7528177	test: 0.7234043	best: 0.7234043 (179)	total: 2m 4s	remaining: 1h 52m 49s
180:	learn: 0.7529956	test: 0.7234610	best: 0.7234610 (180)	total: 2m 4s	remaining: 1h 52m 49s
181:	learn: 0.7531479	test: 0.7236047	best: 0.7236047 (181)	total: 2m 5s	remaining: 1h 52m 47s
182:	learn: 0.7533915	test: 0.7237353	best: 0.7237353 (182)	total: 2m 6s	remaining: 1h 52m 44s
183:	learn: 0.7535113	test: 0.7237933	best: 0.7237933 (183)	total: 2m 6s	remaining: 1h 52m 40s
184:	learn: 0.7537911	test: 0.7240031	best: 0.7240031 (184)	total: 2m 7s	remaining: 1h 52m 37s
185:	learn: 0.7539865	test: 0.7240585	best: 0.7240585 (185)	total: 2m 8s	remaining: 1h 52m 33s
186:	learn: 0.7541518	test: 0.7240774	best: 0.7240

262:	learn: 0.7677829	test: 0.7288045	best: 0.7288045 (262)	total: 3m	remaining: 1h 51m 18s
263:	learn: 0.7680062	test: 0.7288564	best: 0.7288564 (263)	total: 3m 1s	remaining: 1h 51m 15s
264:	learn: 0.7681488	test: 0.7288697	best: 0.7288697 (264)	total: 3m 1s	remaining: 1h 51m 12s
265:	learn: 0.7683101	test: 0.7289423	best: 0.7289423 (265)	total: 3m 2s	remaining: 1h 51m 10s
266:	learn: 0.7684832	test: 0.7289952	best: 0.7289952 (266)	total: 3m 2s	remaining: 1h 51m 6s
267:	learn: 0.7686674	test: 0.7290211	best: 0.7290211 (267)	total: 3m 3s	remaining: 1h 51m 4s
268:	learn: 0.7688106	test: 0.7290681	best: 0.7290681 (268)	total: 3m 4s	remaining: 1h 51m 2s
269:	learn: 0.7689762	test: 0.7291710	best: 0.7291710 (269)	total: 3m 4s	remaining: 1h 50m 59s
270:	learn: 0.7692122	test: 0.7291956	best: 0.7291956 (270)	total: 3m 5s	remaining: 1h 50m 57s
271:	learn: 0.7693802	test: 0.7292595	best: 0.7292595 (271)	total: 3m 6s	remaining: 1h 50m 55s
272:	learn: 0.7695503	test: 0.7293080	best: 0.7293080 (2

348:	learn: 0.7829896	test: 0.7322146	best: 0.7322146 (348)	total: 3m 57s	remaining: 1h 49m 27s
349:	learn: 0.7831607	test: 0.7322069	best: 0.7322146 (348)	total: 3m 58s	remaining: 1h 49m 26s
350:	learn: 0.7833668	test: 0.7322421	best: 0.7322421 (350)	total: 3m 58s	remaining: 1h 49m 29s
351:	learn: 0.7836316	test: 0.7322266	best: 0.7322421 (350)	total: 3m 59s	remaining: 1h 49m 28s
352:	learn: 0.7838018	test: 0.7322568	best: 0.7322568 (352)	total: 4m	remaining: 1h 49m 31s
353:	learn: 0.7840316	test: 0.7323289	best: 0.7323289 (353)	total: 4m 1s	remaining: 1h 49m 37s
354:	learn: 0.7842083	test: 0.7323664	best: 0.7323664 (354)	total: 4m 2s	remaining: 1h 49m 36s
355:	learn: 0.7843137	test: 0.7323538	best: 0.7323664 (354)	total: 4m 2s	remaining: 1h 49m 38s
356:	learn: 0.7845803	test: 0.7323943	best: 0.7323943 (356)	total: 4m 3s	remaining: 1h 49m 38s
357:	learn: 0.7848218	test: 0.7324192	best: 0.7324192 (357)	total: 4m 4s	remaining: 1h 49m 35s
358:	learn: 0.7850474	test: 0.7325148	best: 0.732

...

520:	learn: 0.8133100	test: 0.7359442	best: 0.7359442 (520)	total: 6m 4s	remaining: 1h 50m 36s
521:	learn: 0.8134687	test: 0.7360052	best: 0.7360052 (521)	total: 6m 5s	remaining: 1h 50m 35s
522:	learn: 0.8136165	test: 0.7359895	best: 0.7360052 (521)	total: 6m 6s	remaining: 1h 50m 34s
523:	learn: 0.8138007	test: 0.7360109	best: 0.7360109 (523)	total: 6m 6s	remaining: 1h 50m 33s
524:	learn: 0.8139583	test: 0.7359975	best: 0.7360109 (523)	total: 6m 7s	remaining: 1h 50m 31s
525:	learn: 0.8140868	test: 0.7360034	best: 0.7360109 (523)	total: 6m 8s	remaining: 1h 50m 31s
526:	learn: 0.8142065	test: 0.7360361	best: 0.7360361 (526)	total: 6m 9s	remaining: 1h 50m 33s
527:	learn: 0.8143915	test: 0.7360346	best: 0.7360361 (526)	total: 6m 9s	remaining: 1h 50m 34s
528:	learn: 0.8145025	test: 0.7360812	best: 0.7360812 (528)	total: 6m 10s	remaining: 1h 50m 33s
529:	learn: 0.8146443	test: 0.7360527	best: 0.7360812 (528)	total: 6m 11s	remaining: 1h 50m 32s
530:	learn: 0.8147697	test: 0.7360959	best: 0.73

606:	learn: 0.8259906	test: 0.7369488	best: 0.7369488 (606)	total: 7m 2s	remaining: 1h 49m 3s
607:	learn: 0.8261100	test: 0.7369654	best: 0.7369654 (607)	total: 7m 3s	remaining: 1h 49m 1s
608:	learn: 0.8262515	test: 0.7369907	best: 0.7369907 (608)	total: 7m 4s	remaining: 1h 48m 59s
609:	learn: 0.8263612	test: 0.7370245	best: 0.7370245 (609)	total: 7m 4s	remaining: 1h 48m 58s
610:	learn: 0.8264734	test: 0.7370517	best: 0.7370517 (610)	total: 7m 5s	remaining: 1h 48m 56s
611:	learn: 0.8265919	test: 0.7370883	best: 0.7370883 (611)	total: 7m 6s	remaining: 1h 48m 55s
612:	learn: 0.8267033	test: 0.7371164	best: 0.7371164 (612)	total: 7m 6s	remaining: 1h 48m 53s
613:	learn: 0.8268101	test: 0.7371239	best: 0.7371239 (613)	total: 7m 7s	remaining: 1h 48m 52s
614:	learn: 0.8269405	test: 0.7371607	best: 0.7371607 (614)	total: 7m 8s	remaining: 1h 48m 51s
615:	learn: 0.8271157	test: 0.7371863	best: 0.7371863 (615)	total: 7m 8s	remaining: 1h 48m 50s
616:	learn: 0.8272813	test: 0.7371854	best: 0.737186

692:	learn: 0.8374532	test: 0.7378283	best: 0.7378420 (689)	total: 8m 1s	remaining: 1h 47m 42s
693:	learn: 0.8375412	test: 0.7378512	best: 0.7378512 (693)	total: 8m 1s	remaining: 1h 47m 41s
694:	learn: 0.8376249	test: 0.7378289	best: 0.7378512 (693)	total: 8m 2s	remaining: 1h 47m 40s
695:	learn: 0.8377424	test: 0.7378183	best: 0.7378512 (693)	total: 8m 3s	remaining: 1h 47m 39s
696:	learn: 0.8378611	test: 0.7378296	best: 0.7378512 (693)	total: 8m 3s	remaining: 1h 47m 38s
697:	learn: 0.8379984	test: 0.7378569	best: 0.7378569 (697)	total: 8m 4s	remaining: 1h 47m 38s
698:	learn: 0.8381458	test: 0.7378871	best: 0.7378871 (698)	total: 8m 5s	remaining: 1h 47m 39s
699:	learn: 0.8382706	test: 0.7379021	best: 0.7379021 (699)	total: 8m 6s	remaining: 1h 47m 42s
700:	learn: 0.8383732	test: 0.7379288	best: 0.7379288 (700)	total: 8m 7s	remaining: 1h 47m 44s
701:	learn: 0.8384882	test: 0.7378877	best: 0.7379288 (700)	total: 8m 8s	remaining: 1h 47m 48s
702:	learn: 0.8386152	test: 0.7379012	best: 0.7379

778:	learn: 0.8473164	test: 0.7386061	best: 0.7386302 (755)	total: 9m 2s	remaining: 1h 47m 2s
779:	learn: 0.8474358	test: 0.7386259	best: 0.7386302 (755)	total: 9m 3s	remaining: 1h 47m
780:	learn: 0.8475684	test: 0.7386087	best: 0.7386302 (755)	total: 9m 3s	remaining: 1h 46m 59s
781:	learn: 0.8476898	test: 0.7386295	best: 0.7386302 (755)	total: 9m 4s	remaining: 1h 46m 58s
782:	learn: 0.8478193	test: 0.7386408	best: 0.7386408 (782)	total: 9m 5s	remaining: 1h 47m
783:	learn: 0.8478877	test: 0.7386379	best: 0.7386408 (782)	total: 9m 6s	remaining: 1h 47m
784:	learn: 0.8480321	test: 0.7386375	best: 0.7386408 (782)	total: 9m 6s	remaining: 1h 46m 59s
785:	learn: 0.8481086	test: 0.7386577	best: 0.7386577 (785)	total: 9m 7s	remaining: 1h 46m 59s
786:	learn: 0.8482013	test: 0.7386895	best: 0.7386895 (786)	total: 9m 8s	remaining: 1h 47m
787:	learn: 0.8483263	test: 0.7387146	best: 0.7387146 (787)	total: 9m 9s	remaining: 1h 46m 58s
788:	learn: 0.8484351	test: 0.7387119	best: 0.7387146 (787)	total: 

,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-AUC-mean,train-AUC-std,train-Logloss-mean,train-Logloss-std
0,0.645378,0.008900,0.676433,0.000109,0.650517,0.003221,0.676365,0.000094
1,0.652377,0.004881,0.662545,0.000135,0.658844,0.005411,0.662459,0.000158
2,0.659402,0.007077,0.648726,0.000213,0.663868,0.004092,0.648624,0.000100
3,0.660056,0.005883,0.635389,0.000263,0.668009,0.003763,0.635202,0.000145
4,0.664418,0.007612,0.622999,0.000339,0.674557,0.003714,0.622705,0.000148
5,0.669545,0.005347,0.611357,0.000288,0.678542,0.001853,0.611004,0.000054
6,0.668482,0.004469,0.601175,0.000543,0.677484,0.003142,0.600763,0.000582
7,0.671174,0.003992,0.590973,0.000292,0.679877,0.005341,0.590496,0.000072
8,0.673405,0.001475,0.581327,0.000326,0.682328,0.005521,0.580807,0.000238
9,0.673921,0.000320,0.572521,0.000335,0.683155,0.006793,0.571989,0.000257


In [9]:
parameters = {
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 10000,
    'learning_rate': 0.03,
    'random_seed': 42,
    'use_best_model': False,
    'od_wait': 30,
    'od_type': 'Iter'
}


for train, val in skf.split(X,y):
    temp_ctb_train = ctb.Pool(X.loc[train], y.loc[train])
    temp_ctb_val = ctb.Pool(X.loc[val], y.loc[val])
    temp_model = ctb.train(temp_ctb_train, parameters, logging_level='Verbose', eval_set=temp_ctb_val)

0:	learn: 0.5192277	test: 0.5263602	best: 0.5263602 (0)	total: 202ms	remaining: 33m 40s
1:	learn: 0.6325979	test: 0.6223477	best: 0.6223477 (1)	total: 409ms	remaining: 34m 2s
2:	learn: 0.6491822	test: 0.6413789	best: 0.6413789 (2)	total: 532ms	remaining: 29m 32s
3:	learn: 0.6574625	test: 0.6530961	best: 0.6530961 (3)	total: 754ms	remaining: 31m 23s
4:	learn: 0.6692899	test: 0.6623995	best: 0.6623995 (4)	total: 966ms	remaining: 32m 11s
5:	learn: 0.6761833	test: 0.6668646	best: 0.6668646 (5)	total: 1.21s	remaining: 33m 28s
6:	learn: 0.6781513	test: 0.6706322	best: 0.6706322 (6)	total: 1.5s	remaining: 35m 42s
7:	learn: 0.6787747	test: 0.6699784	best: 0.6706322 (6)	total: 1.81s	remaining: 37m 45s
8:	learn: 0.6799765	test: 0.6698301	best: 0.6706322 (6)	total: 2s	remaining: 37m 6s
9:	learn: 0.6793924	test: 0.6691837	best: 0.6706322 (6)	total: 2.22s	remaining: 36m 53s
10:	learn: 0.6814108	test: 0.6702634	best: 0.6706322 (6)	total: 2.49s	remaining: 37m 45s
11:	learn: 0.6816259	test: 0.6709856	

92:	learn: 0.7304584	test: 0.7131724	best: 0.7131724 (92)	total: 22.5s	remaining: 39m 59s
93:	learn: 0.7310490	test: 0.7136737	best: 0.7136737 (93)	total: 22.8s	remaining: 39m 58s
94:	learn: 0.7316158	test: 0.7138922	best: 0.7138922 (94)	total: 23s	remaining: 39m 53s
95:	learn: 0.7323096	test: 0.7145822	best: 0.7145822 (95)	total: 23.2s	remaining: 39m 51s
96:	learn: 0.7325704	test: 0.7147232	best: 0.7147232 (96)	total: 23.4s	remaining: 39m 48s
97:	learn: 0.7326790	test: 0.7148253	best: 0.7148253 (97)	total: 23.6s	remaining: 39m 46s
98:	learn: 0.7329273	test: 0.7151069	best: 0.7151069 (98)	total: 23.8s	remaining: 39m 43s
99:	learn: 0.7334224	test: 0.7152433	best: 0.7152433 (99)	total: 24.1s	remaining: 39m 43s
100:	learn: 0.7335762	test: 0.7153401	best: 0.7153401 (100)	total: 24.3s	remaining: 39m 41s
101:	learn: 0.7340446	test: 0.7157194	best: 0.7157194 (101)	total: 24.5s	remaining: 39m 37s
102:	learn: 0.7343540	test: 0.7158556	best: 0.7158556 (102)	total: 24.7s	remaining: 39m 34s
103:	l

182:	learn: 0.7540940	test: 0.7252446	best: 0.7252745 (181)	total: 42.4s	remaining: 37m 54s
183:	learn: 0.7542719	test: 0.7254178	best: 0.7254178 (183)	total: 42.6s	remaining: 37m 54s
184:	learn: 0.7545681	test: 0.7254667	best: 0.7254667 (184)	total: 42.8s	remaining: 37m 53s
185:	learn: 0.7548335	test: 0.7257638	best: 0.7257638 (185)	total: 43.1s	remaining: 37m 51s
186:	learn: 0.7551175	test: 0.7258362	best: 0.7258362 (186)	total: 43.3s	remaining: 37m 51s
187:	learn: 0.7553718	test: 0.7260078	best: 0.7260078 (187)	total: 43.5s	remaining: 37m 49s
188:	learn: 0.7555226	test: 0.7261382	best: 0.7261382 (188)	total: 43.7s	remaining: 37m 48s
189:	learn: 0.7556930	test: 0.7261662	best: 0.7261662 (189)	total: 43.9s	remaining: 37m 48s
190:	learn: 0.7558680	test: 0.7263249	best: 0.7263249 (190)	total: 44.2s	remaining: 37m 48s
191:	learn: 0.7560395	test: 0.7264216	best: 0.7264216 (191)	total: 44.4s	remaining: 37m 46s
192:	learn: 0.7561816	test: 0.7264258	best: 0.7264258 (192)	total: 44.6s	remaini

272:	learn: 0.7722402	test: 0.7310469	best: 0.7310469 (272)	total: 1m 2s	remaining: 37m 1s
273:	learn: 0.7724508	test: 0.7312243	best: 0.7312243 (273)	total: 1m 2s	remaining: 37m 1s
274:	learn: 0.7728000	test: 0.7313431	best: 0.7313431 (274)	total: 1m 2s	remaining: 37m
275:	learn: 0.7729351	test: 0.7313699	best: 0.7313699 (275)	total: 1m 3s	remaining: 37m
276:	learn: 0.7731371	test: 0.7314076	best: 0.7314076 (276)	total: 1m 3s	remaining: 36m 59s
277:	learn: 0.7733690	test: 0.7315440	best: 0.7315440 (277)	total: 1m 3s	remaining: 36m 59s
278:	learn: 0.7734570	test: 0.7315853	best: 0.7315853 (278)	total: 1m 3s	remaining: 36m 58s
279:	learn: 0.7737079	test: 0.7316712	best: 0.7316712 (279)	total: 1m 3s	remaining: 36m 58s
280:	learn: 0.7737624	test: 0.7316863	best: 0.7316863 (280)	total: 1m 4s	remaining: 36m 56s
281:	learn: 0.7739752	test: 0.7316876	best: 0.7316876 (281)	total: 1m 4s	remaining: 36m 56s
282:	learn: 0.7740988	test: 0.7316834	best: 0.7316876 (281)	total: 1m 4s	remaining: 36m 56

361:	learn: 0.7896043	test: 0.7343540	best: 0.7343540 (361)	total: 1m 25s	remaining: 37m 43s
362:	learn: 0.7897520	test: 0.7343827	best: 0.7343827 (362)	total: 1m 25s	remaining: 37m 43s
363:	learn: 0.7899948	test: 0.7343565	best: 0.7343827 (362)	total: 1m 25s	remaining: 37m 42s
364:	learn: 0.7903532	test: 0.7343683	best: 0.7343827 (362)	total: 1m 25s	remaining: 37m 43s
365:	learn: 0.7906200	test: 0.7344163	best: 0.7344163 (365)	total: 1m 26s	remaining: 37m 43s
366:	learn: 0.7908065	test: 0.7344889	best: 0.7344889 (366)	total: 1m 26s	remaining: 37m 44s
367:	learn: 0.7908936	test: 0.7345900	best: 0.7345900 (367)	total: 1m 26s	remaining: 37m 44s
368:	learn: 0.7910455	test: 0.7345133	best: 0.7345900 (367)	total: 1m 26s	remaining: 37m 45s
369:	learn: 0.7912070	test: 0.7345303	best: 0.7345900 (367)	total: 1m 27s	remaining: 37m 44s
370:	learn: 0.7914949	test: 0.7344893	best: 0.7345900 (367)	total: 1m 27s	remaining: 37m 44s
371:	learn: 0.7916478	test: 0.7345361	best: 0.7345900 (367)	total: 1m 

...

74:	learn: 0.7227445	test: 0.7093261	best: 0.7093261 (74)	total: 19.2s	remaining: 42m 23s
75:	learn: 0.7232286	test: 0.7099395	best: 0.7099395 (75)	total: 19.5s	remaining: 42m 19s
76:	learn: 0.7238052	test: 0.7104053	best: 0.7104053 (76)	total: 19.7s	remaining: 42m 20s
77:	learn: 0.7240904	test: 0.7106840	best: 0.7106840 (77)	total: 20s	remaining: 42m 29s
78:	learn: 0.7242017	test: 0.7107880	best: 0.7107880 (78)	total: 20.3s	remaining: 42m 24s
79:	learn: 0.7249486	test: 0.7113768	best: 0.7113768 (79)	total: 20.5s	remaining: 42m 19s
80:	learn: 0.7255006	test: 0.7119931	best: 0.7119931 (80)	total: 20.8s	remaining: 42m 29s
81:	learn: 0.7256829	test: 0.7120816	best: 0.7120816 (81)	total: 21.1s	remaining: 42m 32s
82:	learn: 0.7259915	test: 0.7123959	best: 0.7123959 (82)	total: 21.4s	remaining: 42m 39s
83:	learn: 0.7265376	test: 0.7129719	best: 0.7129719 (83)	total: 21.7s	remaining: 42m 45s
84:	learn: 0.7265630	test: 0.7131188	best: 0.7131188 (84)	total: 22s	remaining: 42m 42s
85:	learn: 0.7

164:	learn: 0.7510196	test: 0.7276059	best: 0.7276059 (164)	total: 41s	remaining: 40m 41s
165:	learn: 0.7511581	test: 0.7276393	best: 0.7276393 (165)	total: 41.2s	remaining: 40m 41s
166:	learn: 0.7512370	test: 0.7277466	best: 0.7277466 (166)	total: 41.4s	remaining: 40m 38s
167:	learn: 0.7513480	test: 0.7277740	best: 0.7277740 (167)	total: 41.7s	remaining: 40m 38s
168:	learn: 0.7516599	test: 0.7279008	best: 0.7279008 (168)	total: 41.9s	remaining: 40m 36s
169:	learn: 0.7518548	test: 0.7281130	best: 0.7281130 (169)	total: 42.1s	remaining: 40m 35s
170:	learn: 0.7520531	test: 0.7281055	best: 0.7281130 (169)	total: 42.3s	remaining: 40m 32s
171:	learn: 0.7522029	test: 0.7280722	best: 0.7281130 (169)	total: 42.6s	remaining: 40m 32s
172:	learn: 0.7523821	test: 0.7280897	best: 0.7281130 (169)	total: 42.8s	remaining: 40m 30s
173:	learn: 0.7525494	test: 0.7282221	best: 0.7282221 (173)	total: 43s	remaining: 40m 29s
174:	learn: 0.7528830	test: 0.7283547	best: 0.7283547 (174)	total: 43.2s	remaining: 

254:	learn: 0.7679028	test: 0.7328319	best: 0.7328319 (254)	total: 1m 1s	remaining: 39m 3s
255:	learn: 0.7679572	test: 0.7328003	best: 0.7328319 (254)	total: 1m 1s	remaining: 39m 2s
256:	learn: 0.7681440	test: 0.7328751	best: 0.7328751 (256)	total: 1m 1s	remaining: 39m
257:	learn: 0.7682484	test: 0.7328751	best: 0.7328751 (256)	total: 1m 1s	remaining: 39m
258:	learn: 0.7684100	test: 0.7328789	best: 0.7328789 (258)	total: 1m 2s	remaining: 38m 59s
259:	learn: 0.7685904	test: 0.7328958	best: 0.7328958 (259)	total: 1m 2s	remaining: 38m 58s
260:	learn: 0.7687987	test: 0.7329294	best: 0.7329294 (260)	total: 1m 2s	remaining: 38m 57s
261:	learn: 0.7691883	test: 0.7330057	best: 0.7330057 (261)	total: 1m 2s	remaining: 38m 56s
262:	learn: 0.7693978	test: 0.7330216	best: 0.7330216 (262)	total: 1m 3s	remaining: 38m 56s
263:	learn: 0.7695362	test: 0.7330813	best: 0.7330813 (263)	total: 1m 3s	remaining: 38m 54s
264:	learn: 0.7697785	test: 0.7332252	best: 0.7332252 (264)	total: 1m 3s	remaining: 38m 54

343:	learn: 0.7834852	test: 0.7365450	best: 0.7365450 (343)	total: 1m 21s	remaining: 38m 6s
344:	learn: 0.7835771	test: 0.7365842	best: 0.7365842 (344)	total: 1m 21s	remaining: 38m 5s
345:	learn: 0.7837548	test: 0.7365734	best: 0.7365842 (344)	total: 1m 21s	remaining: 38m 5s
346:	learn: 0.7839278	test: 0.7366580	best: 0.7366580 (346)	total: 1m 22s	remaining: 38m 4s
347:	learn: 0.7840419	test: 0.7368113	best: 0.7368113 (347)	total: 1m 22s	remaining: 38m 4s
348:	learn: 0.7842431	test: 0.7368112	best: 0.7368113 (347)	total: 1m 22s	remaining: 38m 3s
349:	learn: 0.7844800	test: 0.7367512	best: 0.7368113 (347)	total: 1m 22s	remaining: 38m 3s
350:	learn: 0.7846180	test: 0.7367948	best: 0.7368113 (347)	total: 1m 23s	remaining: 38m 2s
351:	learn: 0.7847608	test: 0.7368539	best: 0.7368539 (351)	total: 1m 23s	remaining: 38m 1s
352:	learn: 0.7849389	test: 0.7368937	best: 0.7368937 (352)	total: 1m 23s	remaining: 38m
353:	learn: 0.7852677	test: 0.7368378	best: 0.7368937 (352)	total: 1m 23s	remaining

432:	learn: 0.8005263	test: 0.7383722	best: 0.7383722 (432)	total: 1m 41s	remaining: 37m 25s
433:	learn: 0.8006511	test: 0.7384363	best: 0.7384363 (433)	total: 1m 41s	remaining: 37m 24s
434:	learn: 0.8008163	test: 0.7383952	best: 0.7384363 (433)	total: 1m 42s	remaining: 37m 23s
435:	learn: 0.8009972	test: 0.7383628	best: 0.7384363 (433)	total: 1m 42s	remaining: 37m 23s
436:	learn: 0.8010641	test: 0.7384016	best: 0.7384363 (433)	total: 1m 42s	remaining: 37m 23s
437:	learn: 0.8012594	test: 0.7384172	best: 0.7384363 (433)	total: 1m 42s	remaining: 37m 22s
438:	learn: 0.8014704	test: 0.7384646	best: 0.7384646 (438)	total: 1m 42s	remaining: 37m 22s
439:	learn: 0.8017036	test: 0.7384256	best: 0.7384646 (438)	total: 1m 43s	remaining: 37m 21s
440:	learn: 0.8018337	test: 0.7384277	best: 0.7384646 (438)	total: 1m 43s	remaining: 37m 21s
441:	learn: 0.8019374	test: 0.7383919	best: 0.7384646 (438)	total: 1m 43s	remaining: 37m 20s
442:	learn: 0.8021397	test: 0.7384067	best: 0.7384646 (438)	total: 1m 

In [10]:
np.mean([0.7374885133, 0.7405642612, 0.7391216238])

0.7390581327666667

In [13]:
parameters = {
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 1000,
    'learning_rate': 0.03,
    'random_seed': 42,
    'use_best_model': False,
    'od_wait': 30,
    'od_type': 'Iter'
}

model = CatBoostClassifier(**parameters)

In [14]:
model.fit(X, y)

y_pred_ctb_model = model.predict_proba(test[features])[:, 1]

test_df = test[['Unnamed: 0', '0']]

test_df['0'] = y_pred_ctb_model

test_df.to_csv('CatBoost_tunned.csv', index=False)

0:	learn: 0.6163643	total: 317ms	remaining: 5m 17s
1:	learn: 0.6493309	total: 648ms	remaining: 5m 23s
2:	learn: 0.6524529	total: 963ms	remaining: 5m 20s
3:	learn: 0.6583662	total: 1.3s	remaining: 5m 23s
4:	learn: 0.6636004	total: 1.62s	remaining: 5m 21s
5:	learn: 0.6734287	total: 1.93s	remaining: 5m 19s
6:	learn: 0.6715980	total: 2.25s	remaining: 5m 19s
7:	learn: 0.6712531	total: 2.59s	remaining: 5m 21s
8:	learn: 0.6734392	total: 2.91s	remaining: 5m 20s
9:	learn: 0.6749332	total: 3.25s	remaining: 5m 21s
10:	learn: 0.6775505	total: 3.56s	remaining: 5m 20s
11:	learn: 0.6794968	total: 3.89s	remaining: 5m 20s
12:	learn: 0.6825024	total: 4.2s	remaining: 5m 18s
13:	learn: 0.6839480	total: 4.53s	remaining: 5m 19s
14:	learn: 0.6839940	total: 4.88s	remaining: 5m 20s
15:	learn: 0.6833155	total: 5.21s	remaining: 5m 20s
16:	learn: 0.6837776	total: 5.55s	remaining: 5m 20s
17:	learn: 0.6836370	total: 5.85s	remaining: 5m 19s
18:	learn: 0.6848274	total: 6.2s	remaining: 5m 19s
19:	learn: 0.6860832	tota

158:	learn: 0.7444210	total: 57.9s	remaining: 5m 6s
159:	learn: 0.7445895	total: 58.3s	remaining: 5m 6s
160:	learn: 0.7448319	total: 58.7s	remaining: 5m 5s
161:	learn: 0.7450446	total: 59s	remaining: 5m 5s
162:	learn: 0.7452043	total: 59.5s	remaining: 5m 5s
163:	learn: 0.7452611	total: 60s	remaining: 5m 5s
164:	learn: 0.7454945	total: 1m	remaining: 5m 5s
165:	learn: 0.7456864	total: 1m	remaining: 5m 5s
166:	learn: 0.7459791	total: 1m 1s	remaining: 5m 4s
167:	learn: 0.7459898	total: 1m 1s	remaining: 5m 4s
168:	learn: 0.7461477	total: 1m 2s	remaining: 5m 5s
169:	learn: 0.7462369	total: 1m 2s	remaining: 5m 5s
170:	learn: 0.7464292	total: 1m 2s	remaining: 5m 5s
171:	learn: 0.7465936	total: 1m 3s	remaining: 5m 5s
172:	learn: 0.7467544	total: 1m 3s	remaining: 5m 4s
173:	learn: 0.7469714	total: 1m 4s	remaining: 5m 4s
174:	learn: 0.7471319	total: 1m 4s	remaining: 5m 4s
175:	learn: 0.7473529	total: 1m 5s	remaining: 5m 4s
176:	learn: 0.7473769	total: 1m 5s	remaining: 5m 4s
177:	learn: 0.7475036	

312:	learn: 0.7676118	total: 1m 59s	remaining: 4m 21s
313:	learn: 0.7677066	total: 1m 59s	remaining: 4m 21s
314:	learn: 0.7677735	total: 1m 59s	remaining: 4m 20s
315:	learn: 0.7678604	total: 2m	remaining: 4m 20s
316:	learn: 0.7680284	total: 2m	remaining: 4m 20s
317:	learn: 0.7681272	total: 2m 1s	remaining: 4m 19s
318:	learn: 0.7681762	total: 2m 1s	remaining: 4m 19s
319:	learn: 0.7683780	total: 2m 1s	remaining: 4m 19s
320:	learn: 0.7684429	total: 2m 2s	remaining: 4m 18s
321:	learn: 0.7685576	total: 2m 2s	remaining: 4m 18s
322:	learn: 0.7686476	total: 2m 3s	remaining: 4m 18s
323:	learn: 0.7687664	total: 2m 3s	remaining: 4m 17s
324:	learn: 0.7689090	total: 2m 4s	remaining: 4m 17s
325:	learn: 0.7690627	total: 2m 4s	remaining: 4m 17s
326:	learn: 0.7691984	total: 2m 4s	remaining: 4m 17s
327:	learn: 0.7693879	total: 2m 5s	remaining: 4m 16s
328:	learn: 0.7695488	total: 2m 5s	remaining: 4m 16s
329:	learn: 0.7697179	total: 2m 5s	remaining: 4m 15s
330:	learn: 0.7698267	total: 2m 6s	remaining: 4m 

466:	learn: 0.7878563	total: 2m 56s	remaining: 3m 21s
467:	learn: 0.7879289	total: 2m 57s	remaining: 3m 21s
468:	learn: 0.7880692	total: 2m 57s	remaining: 3m 21s
469:	learn: 0.7882171	total: 2m 57s	remaining: 3m 20s
470:	learn: 0.7883239	total: 2m 58s	remaining: 3m 20s
471:	learn: 0.7884528	total: 2m 58s	remaining: 3m 19s
472:	learn: 0.7885358	total: 2m 58s	remaining: 3m 19s
473:	learn: 0.7886349	total: 2m 59s	remaining: 3m 18s
474:	learn: 0.7887476	total: 2m 59s	remaining: 3m 18s
475:	learn: 0.7888620	total: 2m 59s	remaining: 3m 18s
476:	learn: 0.7890277	total: 3m	remaining: 3m 17s
477:	learn: 0.7891338	total: 3m	remaining: 3m 17s
478:	learn: 0.7892336	total: 3m	remaining: 3m 16s
479:	learn: 0.7893651	total: 3m 1s	remaining: 3m 16s
480:	learn: 0.7894851	total: 3m 1s	remaining: 3m 15s
481:	learn: 0.7895838	total: 3m 1s	remaining: 3m 15s
482:	learn: 0.7896982	total: 3m 2s	remaining: 3m 15s
483:	learn: 0.7898394	total: 3m 2s	remaining: 3m 14s
484:	learn: 0.7900833	total: 3m 2s	remaining:

619:	learn: 0.8058853	total: 3m 49s	remaining: 2m 20s
620:	learn: 0.8060030	total: 3m 49s	remaining: 2m 20s
621:	learn: 0.8060570	total: 3m 50s	remaining: 2m 19s
622:	learn: 0.8061400	total: 3m 50s	remaining: 2m 19s
623:	learn: 0.8062890	total: 3m 50s	remaining: 2m 19s
624:	learn: 0.8064080	total: 3m 51s	remaining: 2m 18s
625:	learn: 0.8064798	total: 3m 51s	remaining: 2m 18s
626:	learn: 0.8066057	total: 3m 51s	remaining: 2m 17s
627:	learn: 0.8067411	total: 3m 52s	remaining: 2m 17s
628:	learn: 0.8068813	total: 3m 52s	remaining: 2m 17s
629:	learn: 0.8070247	total: 3m 52s	remaining: 2m 16s
630:	learn: 0.8071455	total: 3m 53s	remaining: 2m 16s
631:	learn: 0.8072266	total: 3m 53s	remaining: 2m 16s
632:	learn: 0.8073892	total: 3m 53s	remaining: 2m 15s
633:	learn: 0.8074889	total: 3m 54s	remaining: 2m 15s
634:	learn: 0.8076426	total: 3m 54s	remaining: 2m 14s
635:	learn: 0.8077307	total: 3m 54s	remaining: 2m 14s
636:	learn: 0.8078135	total: 3m 55s	remaining: 2m 14s
637:	learn: 0.8079121	total:

772:	learn: 0.8218935	total: 4m 43s	remaining: 1m 23s
773:	learn: 0.8219359	total: 4m 43s	remaining: 1m 22s
774:	learn: 0.8220071	total: 4m 43s	remaining: 1m 22s
775:	learn: 0.8221315	total: 4m 44s	remaining: 1m 22s
776:	learn: 0.8222468	total: 4m 44s	remaining: 1m 21s
777:	learn: 0.8223433	total: 4m 44s	remaining: 1m 21s
778:	learn: 0.8224366	total: 4m 45s	remaining: 1m 20s
779:	learn: 0.8224733	total: 4m 45s	remaining: 1m 20s
780:	learn: 0.8225438	total: 4m 45s	remaining: 1m 20s
781:	learn: 0.8226509	total: 4m 46s	remaining: 1m 19s
782:	learn: 0.8227845	total: 4m 46s	remaining: 1m 19s
783:	learn: 0.8230254	total: 4m 46s	remaining: 1m 19s
784:	learn: 0.8230767	total: 4m 47s	remaining: 1m 18s
785:	learn: 0.8232380	total: 4m 47s	remaining: 1m 18s
786:	learn: 0.8232961	total: 4m 47s	remaining: 1m 17s
787:	learn: 0.8234300	total: 4m 48s	remaining: 1m 17s
788:	learn: 0.8234571	total: 4m 48s	remaining: 1m 17s
789:	learn: 0.8235675	total: 4m 48s	remaining: 1m 16s
790:	learn: 0.8236534	total:

928:	learn: 0.8356052	total: 5m 39s	remaining: 26s
929:	learn: 0.8356963	total: 5m 40s	remaining: 25.6s
930:	learn: 0.8357668	total: 5m 40s	remaining: 25.3s
931:	learn: 0.8357752	total: 5m 41s	remaining: 24.9s
932:	learn: 0.8358702	total: 5m 41s	remaining: 24.5s
933:	learn: 0.8359718	total: 5m 42s	remaining: 24.2s
934:	learn: 0.8360313	total: 5m 42s	remaining: 23.8s
935:	learn: 0.8361030	total: 5m 42s	remaining: 23.4s
936:	learn: 0.8361825	total: 5m 43s	remaining: 23.1s
937:	learn: 0.8362656	total: 5m 43s	remaining: 22.7s
938:	learn: 0.8363807	total: 5m 44s	remaining: 22.4s
939:	learn: 0.8364460	total: 5m 44s	remaining: 22s
940:	learn: 0.8365485	total: 5m 44s	remaining: 21.6s
941:	learn: 0.8366229	total: 5m 45s	remaining: 21.3s
942:	learn: 0.8366847	total: 5m 45s	remaining: 20.9s
943:	learn: 0.8368406	total: 5m 45s	remaining: 20.5s
944:	learn: 0.8368847	total: 5m 46s	remaining: 20.2s
945:	learn: 0.8369690	total: 5m 46s	remaining: 19.8s
946:	learn: 0.8370682	total: 5m 47s	remaining: 19.

## Ensemble

Validation : 0.7424155654445643  
LB : 0.75077948

In [26]:
parameters_lgb = {
    'objective': 'binary',
    'learning_rate': 0.01,
    'max_depth': 8,
    'num_threads': 4,
    'metric': 'auc',
    'n_estimators': 580,
    
        #regularization
    'colsample_bytree': 0.6,
    'subsample': 1,
    'subsample_freq': 2,
    'min_data_in_leaf': 20,
}

lg = LGBMClassifier(**parameters_lgb)

In [27]:
parameters_ct = {
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 1000,
    'learning_rate': 0.03,
    'random_seed': 42,
    'use_best_model': False,
    'od_wait': 30,
    'od_type': 'Iter'
}

ct = CatBoostClassifier(**parameters_ct)

In [31]:
lgb_predictions = []
ctb_predictions = []

for train, val in skf.split(X, y):
    lg.fit(X.iloc[train], y[train])
    ct.fit(X.iloc[train], y[train])
    lgb_predictions.append([y[val], lg.predict_proba(X.iloc[val])[:,1]])
    ctb_predictions.append([y[val], ct.predict_proba(X.iloc[val])[:,1]])

0:	learn: 0.5192277	total: 206ms	remaining: 3m 26s
1:	learn: 0.6325979	total: 408ms	remaining: 3m 23s
2:	learn: 0.6491822	total: 559ms	remaining: 3m 5s
3:	learn: 0.6574625	total: 881ms	remaining: 3m 39s
4:	learn: 0.6692899	total: 1.08s	remaining: 3m 35s
5:	learn: 0.6761833	total: 1.3s	remaining: 3m 35s
6:	learn: 0.6781513	total: 1.5s	remaining: 3m 33s
7:	learn: 0.6787747	total: 1.87s	remaining: 3m 52s
8:	learn: 0.6799765	total: 2.12s	remaining: 3m 53s
9:	learn: 0.6793924	total: 2.45s	remaining: 4m 3s
10:	learn: 0.6814108	total: 2.68s	remaining: 4m
11:	learn: 0.6816259	total: 2.91s	remaining: 3m 59s
12:	learn: 0.6813983	total: 3.22s	remaining: 4m 4s
13:	learn: 0.6830934	total: 3.49s	remaining: 4m 5s
14:	learn: 0.6844007	total: 3.68s	remaining: 4m 1s
15:	learn: 0.6863648	total: 4.05s	remaining: 4m 9s
16:	learn: 0.6884869	total: 4.26s	remaining: 4m 6s
17:	learn: 0.6904722	total: 4.49s	remaining: 4m 4s
18:	learn: 0.6915373	total: 4.7s	remaining: 4m 2s
19:	learn: 0.6914926	total: 4.92s	rema

158:	learn: 0.7494984	total: 36.6s	remaining: 3m 13s
159:	learn: 0.7497792	total: 36.8s	remaining: 3m 13s
160:	learn: 0.7500130	total: 37s	remaining: 3m 13s
161:	learn: 0.7501593	total: 37.3s	remaining: 3m 12s
162:	learn: 0.7503284	total: 37.5s	remaining: 3m 12s
163:	learn: 0.7505788	total: 37.7s	remaining: 3m 11s
164:	learn: 0.7506607	total: 37.9s	remaining: 3m 11s
165:	learn: 0.7509139	total: 38.1s	remaining: 3m 11s
166:	learn: 0.7510568	total: 38.3s	remaining: 3m 10s
167:	learn: 0.7512087	total: 38.5s	remaining: 3m 10s
168:	learn: 0.7515113	total: 38.7s	remaining: 3m 10s
169:	learn: 0.7516937	total: 38.9s	remaining: 3m 10s
170:	learn: 0.7518514	total: 39.1s	remaining: 3m 9s
171:	learn: 0.7519529	total: 39.3s	remaining: 3m 9s
172:	learn: 0.7520637	total: 39.6s	remaining: 3m 9s
173:	learn: 0.7522306	total: 39.8s	remaining: 3m 8s
174:	learn: 0.7525601	total: 40s	remaining: 3m 8s
175:	learn: 0.7528144	total: 40.2s	remaining: 3m 8s
176:	learn: 0.7530363	total: 40.4s	remaining: 3m 7s
177:

315:	learn: 0.7805994	total: 1m 10s	remaining: 2m 31s
316:	learn: 0.7807486	total: 1m 10s	remaining: 2m 31s
317:	learn: 0.7810187	total: 1m 10s	remaining: 2m 31s
318:	learn: 0.7813563	total: 1m 10s	remaining: 2m 31s
319:	learn: 0.7816131	total: 1m 11s	remaining: 2m 31s
320:	learn: 0.7816926	total: 1m 11s	remaining: 2m 31s
321:	learn: 0.7818491	total: 1m 11s	remaining: 2m 30s
322:	learn: 0.7820043	total: 1m 11s	remaining: 2m 30s
323:	learn: 0.7822669	total: 1m 12s	remaining: 2m 30s
324:	learn: 0.7824285	total: 1m 12s	remaining: 2m 30s
325:	learn: 0.7826646	total: 1m 12s	remaining: 2m 30s
326:	learn: 0.7827477	total: 1m 12s	remaining: 2m 29s
327:	learn: 0.7828945	total: 1m 13s	remaining: 2m 29s
328:	learn: 0.7831117	total: 1m 13s	remaining: 2m 29s
329:	learn: 0.7832542	total: 1m 13s	remaining: 2m 29s
330:	learn: 0.7835439	total: 1m 13s	remaining: 2m 28s
331:	learn: 0.7837361	total: 1m 13s	remaining: 2m 28s
332:	learn: 0.7838499	total: 1m 14s	remaining: 2m 28s
333:	learn: 0.7840241	total:

468:	learn: 0.8096341	total: 1m 46s	remaining: 2m
469:	learn: 0.8098434	total: 1m 46s	remaining: 2m
470:	learn: 0.8099957	total: 1m 47s	remaining: 2m
471:	learn: 0.8101391	total: 1m 47s	remaining: 2m
472:	learn: 0.8103909	total: 1m 47s	remaining: 1m 59s
473:	learn: 0.8105501	total: 1m 47s	remaining: 1m 59s
474:	learn: 0.8106611	total: 1m 48s	remaining: 1m 59s
475:	learn: 0.8107509	total: 1m 48s	remaining: 1m 59s
476:	learn: 0.8109244	total: 1m 48s	remaining: 1m 59s
477:	learn: 0.8110313	total: 1m 48s	remaining: 1m 58s
478:	learn: 0.8112785	total: 1m 49s	remaining: 1m 58s
479:	learn: 0.8114167	total: 1m 49s	remaining: 1m 58s
480:	learn: 0.8115669	total: 1m 49s	remaining: 1m 58s
481:	learn: 0.8117669	total: 1m 49s	remaining: 1m 57s
482:	learn: 0.8118758	total: 1m 49s	remaining: 1m 57s
483:	learn: 0.8119441	total: 1m 50s	remaining: 1m 57s
484:	learn: 0.8120593	total: 1m 50s	remaining: 1m 57s
485:	learn: 0.8122892	total: 1m 50s	remaining: 1m 57s
486:	learn: 0.8124026	total: 1m 50s	remainin

622:	learn: 0.8319491	total: 2m 22s	remaining: 1m 26s
623:	learn: 0.8320852	total: 2m 23s	remaining: 1m 26s
624:	learn: 0.8321584	total: 2m 23s	remaining: 1m 26s
625:	learn: 0.8323249	total: 2m 23s	remaining: 1m 25s
626:	learn: 0.8324040	total: 2m 23s	remaining: 1m 25s
627:	learn: 0.8325872	total: 2m 24s	remaining: 1m 25s
628:	learn: 0.8327071	total: 2m 24s	remaining: 1m 25s
629:	learn: 0.8328684	total: 2m 24s	remaining: 1m 24s
630:	learn: 0.8330795	total: 2m 24s	remaining: 1m 24s
631:	learn: 0.8332178	total: 2m 25s	remaining: 1m 24s
632:	learn: 0.8333434	total: 2m 25s	remaining: 1m 24s
633:	learn: 0.8335667	total: 2m 25s	remaining: 1m 23s
634:	learn: 0.8337567	total: 2m 25s	remaining: 1m 23s
635:	learn: 0.8339720	total: 2m 25s	remaining: 1m 23s
636:	learn: 0.8340381	total: 2m 26s	remaining: 1m 23s
637:	learn: 0.8340850	total: 2m 26s	remaining: 1m 23s
638:	learn: 0.8342215	total: 2m 26s	remaining: 1m 22s
639:	learn: 0.8343327	total: 2m 26s	remaining: 1m 22s
640:	learn: 0.8345159	total:

...

335:	learn: 0.7820793	total: 1m 20s	remaining: 2m 39s
336:	learn: 0.7822897	total: 1m 21s	remaining: 2m 39s
337:	learn: 0.7824246	total: 1m 21s	remaining: 2m 39s
338:	learn: 0.7824604	total: 1m 21s	remaining: 2m 39s
339:	learn: 0.7826982	total: 1m 21s	remaining: 2m 38s
340:	learn: 0.7829590	total: 1m 22s	remaining: 2m 38s
341:	learn: 0.7831986	total: 1m 22s	remaining: 2m 38s
342:	learn: 0.7834335	total: 1m 22s	remaining: 2m 37s
343:	learn: 0.7834852	total: 1m 22s	remaining: 2m 37s
344:	learn: 0.7835771	total: 1m 22s	remaining: 2m 37s
345:	learn: 0.7837548	total: 1m 23s	remaining: 2m 37s
346:	learn: 0.7839278	total: 1m 23s	remaining: 2m 36s
347:	learn: 0.7840419	total: 1m 23s	remaining: 2m 36s
348:	learn: 0.7842431	total: 1m 23s	remaining: 2m 36s
349:	learn: 0.7844800	total: 1m 24s	remaining: 2m 36s
350:	learn: 0.7846180	total: 1m 24s	remaining: 2m 35s
351:	learn: 0.7847608	total: 1m 24s	remaining: 2m 35s
352:	learn: 0.7849389	total: 1m 24s	remaining: 2m 35s
353:	learn: 0.7852677	total:

488:	learn: 0.8099379	total: 1m 57s	remaining: 2m 2s
489:	learn: 0.8101281	total: 1m 57s	remaining: 2m 2s
490:	learn: 0.8102410	total: 1m 57s	remaining: 2m 1s
491:	learn: 0.8103737	total: 1m 57s	remaining: 2m 1s
492:	learn: 0.8104819	total: 1m 57s	remaining: 2m 1s
493:	learn: 0.8107594	total: 1m 58s	remaining: 2m
494:	learn: 0.8109120	total: 1m 58s	remaining: 2m
495:	learn: 0.8110605	total: 1m 58s	remaining: 2m
496:	learn: 0.8111492	total: 1m 58s	remaining: 2m
497:	learn: 0.8113482	total: 1m 59s	remaining: 2m
498:	learn: 0.8115423	total: 1m 59s	remaining: 1m 59s
499:	learn: 0.8118237	total: 1m 59s	remaining: 1m 59s
500:	learn: 0.8119762	total: 1m 59s	remaining: 1m 59s
501:	learn: 0.8121313	total: 1m 59s	remaining: 1m 58s
502:	learn: 0.8123038	total: 2m	remaining: 1m 58s
503:	learn: 0.8124621	total: 2m	remaining: 1m 58s
504:	learn: 0.8126854	total: 2m	remaining: 1m 58s
505:	learn: 0.8128510	total: 2m	remaining: 1m 57s
506:	learn: 0.8129971	total: 2m	remaining: 1m 57s
507:	learn: 0.81320

642:	learn: 0.8319303	total: 2m 32s	remaining: 1m 24s
643:	learn: 0.8321306	total: 2m 33s	remaining: 1m 24s
644:	learn: 0.8322509	total: 2m 33s	remaining: 1m 24s
645:	learn: 0.8323431	total: 2m 33s	remaining: 1m 24s
646:	learn: 0.8324220	total: 2m 33s	remaining: 1m 23s
647:	learn: 0.8325112	total: 2m 34s	remaining: 1m 23s
648:	learn: 0.8327287	total: 2m 34s	remaining: 1m 23s
649:	learn: 0.8328630	total: 2m 34s	remaining: 1m 23s
650:	learn: 0.8330240	total: 2m 34s	remaining: 1m 22s
651:	learn: 0.8331403	total: 2m 34s	remaining: 1m 22s
652:	learn: 0.8332544	total: 2m 35s	remaining: 1m 22s
653:	learn: 0.8333631	total: 2m 35s	remaining: 1m 22s
654:	learn: 0.8335443	total: 2m 35s	remaining: 1m 21s
655:	learn: 0.8336315	total: 2m 35s	remaining: 1m 21s
656:	learn: 0.8337526	total: 2m 36s	remaining: 1m 21s
657:	learn: 0.8339005	total: 2m 36s	remaining: 1m 21s
658:	learn: 0.8340388	total: 2m 36s	remaining: 1m 20s
659:	learn: 0.8340836	total: 2m 36s	remaining: 1m 20s
660:	learn: 0.8342593	total:

798:	learn: 0.8499046	total: 3m 10s	remaining: 47.9s
799:	learn: 0.8500148	total: 3m 10s	remaining: 47.7s
800:	learn: 0.8501162	total: 3m 10s	remaining: 47.4s
801:	learn: 0.8502187	total: 3m 11s	remaining: 47.2s
802:	learn: 0.8503273	total: 3m 11s	remaining: 46.9s
803:	learn: 0.8504810	total: 3m 11s	remaining: 46.7s
804:	learn: 0.8505497	total: 3m 11s	remaining: 46.4s
805:	learn: 0.8506506	total: 3m 11s	remaining: 46.2s
806:	learn: 0.8506720	total: 3m 12s	remaining: 45.9s
807:	learn: 0.8508428	total: 3m 12s	remaining: 45.7s
808:	learn: 0.8510673	total: 3m 12s	remaining: 45.4s
809:	learn: 0.8511931	total: 3m 12s	remaining: 45.2s
810:	learn: 0.8513026	total: 3m 12s	remaining: 45s
811:	learn: 0.8513913	total: 3m 13s	remaining: 44.7s
812:	learn: 0.8514410	total: 3m 13s	remaining: 44.5s
813:	learn: 0.8514871	total: 3m 13s	remaining: 44.2s
814:	learn: 0.8515782	total: 3m 13s	remaining: 44s
815:	learn: 0.8516459	total: 3m 13s	remaining: 43.7s
816:	learn: 0.8517187	total: 3m 14s	remaining: 43.

955:	learn: 0.8654640	total: 3m 43s	remaining: 10.3s
956:	learn: 0.8655287	total: 3m 44s	remaining: 10.1s
957:	learn: 0.8657162	total: 3m 44s	remaining: 9.83s
958:	learn: 0.8658143	total: 3m 44s	remaining: 9.6s
959:	learn: 0.8658586	total: 3m 44s	remaining: 9.36s
960:	learn: 0.8659596	total: 3m 44s	remaining: 9.13s
961:	learn: 0.8660725	total: 3m 45s	remaining: 8.89s
962:	learn: 0.8662007	total: 3m 45s	remaining: 8.66s
963:	learn: 0.8662797	total: 3m 45s	remaining: 8.42s
964:	learn: 0.8663957	total: 3m 45s	remaining: 8.19s
965:	learn: 0.8665838	total: 3m 45s	remaining: 7.95s
966:	learn: 0.8666911	total: 3m 46s	remaining: 7.72s
967:	learn: 0.8667345	total: 3m 46s	remaining: 7.48s
968:	learn: 0.8668237	total: 3m 46s	remaining: 7.25s
969:	learn: 0.8669823	total: 3m 46s	remaining: 7.01s
970:	learn: 0.8670392	total: 3m 47s	remaining: 6.78s
971:	learn: 0.8671327	total: 3m 47s	remaining: 6.55s
972:	learn: 0.8672325	total: 3m 47s	remaining: 6.31s
973:	learn: 0.8673680	total: 3m 47s	remaining: 

In [43]:
# Weighted average
np.mean([roc_auc_score(lg_p[0], lg_p[1]*0.5 + ct_p[1]*0.5) for lg_p, ct_p in zip(lgb_predictions, ctb_predictions)])

0.7424155654445643

In [52]:
avrg_result = y_pred_lgb_model*0.5 + y_pred_ctb_model*0.5

test_df = test[['Unnamed: 0', '0']]

test_df['0'] = y_pred_ctb_model

test_df.to_csv('Ensemble_LGBM_CatBoost.csv', index=False)

## Stacking
Validation : 0.73404229  
LB : 0.75706025

In [61]:
parameters_xgb = {
    #General Parameters
    'silent': 1,
    "nthread": 4,
    
    #Booster Parameters
    'learning_rate': 0.1,
    'max_depth': 4,
    'min_child_weight': 5,
    'gamma': 0.1,
    'subsample': 1,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'colsample_bytree': 1,
    
    #Learning Task Parameters
    'objective': 'reg:logistic',
    "seed": 42,
    "eval_metric": 'auc'
}


xg = XGBClassifier(**parameters_xgb)

In [62]:
xbg_predictions = []

for train, val in skf.split(X, y):
    xg.fit(X.iloc[train], y[train])
    xbg_predictions.append([y[val], xg.predict_proba(X.iloc[val])[:,1]])

In [85]:
X["xgb_preds"] = 0
X["lgb_preds"] = 0
X["ctb_preds"] = 0

for i, fold in enumerate(skf.split(X, y)):
    train, val = fold[0], fold[1]
    X.loc[val, 'xgb_preds'] = xbg_predictions[i][1]
    X.loc[val, 'lgb_preds'] = lgb_predictions[i][1]
    X.loc[val, 'ctb_preds'] = ctb_predictions[i][1]

In [86]:
test["xgb_preds"] = y_pred_xgb_model
test["lgb_preds"] = y_pred_lgb_model
test["ctb_preds"] = y_pred_ctb_model

In [95]:
lgb_train = lgb.Dataset(X, label=y, free_raw_data=False)
check_train_score(parameters_lgb, lgb_train, X, y, skf, 580)

array([0.85325607, 0.73404229])

In [109]:
model_lbg = LGBMClassifier(**parameters_lgb)

model_lbg.fit(X, y)

y_pred_stack = model_lbg.predict_proba(test[features])[:, 1]

test_df = test[['Unnamed: 0', '0']]

test_df['0'] = y_pred_stack

test_df.to_csv('stacking.csv', index=False)